In [2]:
import pandas as pd
import numpy as np
import duckdb
import time

import c0

In [2]:
import importlib
importlib.reload(c0)

<module 'c0' from '/home/ryfran/ValidProt/pipeline/c0_preprocessing/c0.py'>

In [6]:
con = c0.connect_db('/mnt/c/Users/Ryan/Documents/database')

Connecting to database...
Connection established! Execution time: 30.534417390823364 seconds


In [4]:
c0.build_validprot(con = con)

Constructing vp_taxa_pairs...
Finished constructing vp_taxa_pairs. Execution time: 0.07487940788269043 seconds
Constructing vp_taxa...
Finished constructing vp_taxa. Execution time: 0.1631009578704834 seconds
Filtering on ogt and 16S sequence parameters...
Finished filtering. Execution time: 0.020915508270263672 seconds
Constructing vp_protein_pairs...
Finished constructing vp_protein_pairs. Execution time: 22.390434980392456 seconds
Constructing vp_proteins...
Finished constructing vp_proteins. Execution time: 36.234153747558594 seconds
Constructing final dataset...
Finishing up...
Finished. Total execution time: 381.0085434913635 seconds


In [5]:
con.execute("""SELECT * FROM vp_final LIMIT 1""").df()

ConnectionException: Connection Error: Connection has already been closed

## Making sample dataset for unit testing

In [7]:
con.execute("""CREATE OR REPLACE TABLE vp_sample AS SELECT * FROM vp_final USING SAMPLE 100""")
samp_df = con.execute("""SELECT * FROM vp_sample""").df()
samp_protein_pairs = con.execute("""SELECT * FROM protein_pairs WHERE prot_pair_index IN 
(SELECT prot_pair_index FROM vp_sample)""").df()
samp_proteins = con.execute("""SELECT * FROM vp_proteins
WHERE vp_proteins.protein_index IN (SELECT DISTINCT thermo_protein_index FROM vp_sample)
OR vp_proteins.protein_index IN (SELECT DISTINCT meso_protein_index FROM vp_sample)""").df()
samp_taxa_pairs = con.execute("""SELECT * FROM vp_taxa_pairs WHERE taxa_pair_index IN (SELECT taxa_pair_index FROM vp_sample)""").df()
samp_taxa = con.execute("""SELECT * FROM vp_taxa WHERE taxa_index in (SELECT meso_index FROM vp_sample)
OR taxa_index in (SELECT thermo_index FROM vp_sample)""").df()

In [8]:
con.execute("""CREATE OR REPLACE TABLE rand_pairs AS SELECT * FROM protein_pairs USING SAMPLE 100""")
rand_protein_pairs = con.execute("""SELECT * FROM rand_pairs""").df()
rand_proteins = con.execute("""SELECT * FROM proteins WHERE protein_int_index IN (SELECT DISTINCT meso_protein_int_index FROM rand_pairs)
OR protein_int_index IN (SELECT DISTINCT thermo_protein_int_index FROM rand_pairs)""").df()
rand_taxa_pairs = con.execute("""SELECT * FROM taxa_pairs WHERE taxa_pair_index IN (SELECT taxa_pair_index FROM rand_pairs)""").df()
rand_taxa = con.execute("""SELECT * FROM taxa WHERE taxa_index in (SELECT meso_index FROM rand_pairs)
OR taxa_index in (SELECT thermo_index FROM rand_pairs)""").df()

In [9]:
test_protein_pairs = pd.concat([samp_protein_pairs, rand_protein_pairs], axis = 0)
test_proteins = pd.concat([samp_proteins, rand_proteins], axis = 0)
test_taxa = pd.concat([samp_taxa, rand_taxa], axis = 0)
test_taxa_pairs = pd.concat([samp_taxa_pairs, rand_taxa_pairs], axis = 0)


In [35]:
for df in [test_protein_pairs, test_proteins, test_taxa, test_taxa_pairs]:
    print(df.shape)

(200, 21)
(395, 6)
(257, 12)
(199, 17)


In [10]:
con.commit()
con.close()

In [11]:
conn = duckdb.connect('validprot_testing', read_only = False)

In [12]:
conn.execute("""CREATE TABLE protein_pairs AS SELECT * FROM test_protein_pairs""")
conn.execute("""CREATE TABLE proteins AS SELECT * FROM test_proteins""")
conn.execute("""CREATE TABLE taxa_pairs AS SELECT * FROM test_taxa_pairs""")
conn.execute("""CREATE TABLE taxa AS SELECT * FROM test_taxa""")

In [ ]:
con.commit()
con.close()

In [43]:
conn.execute("""SELECT TABLE_NAME
FROM INFORMATION_SCHEMA.TABLES
WHERE TABLE_TYPE='BASE TABLE'""").df()

,table_name
0,protein_pairs
1,proteins
2,taxa_pairs
3,taxa


In [14]:
samp_taxa = con.execute("""SELECT * FROM vp_taxa_pairs WHERE taxa_pair_index IN (SELECT taxa_pair_index FROM vp_final)""").df()
samp_taxa.head()

,thermo_index,meso_index,local_gap_compressed_percent_id,scaled_local_query_percent_id,scaled_local_symmetric_percent_id,local_E_value,query_align_start,query_align_end,subject_align_end,subject_align_start,query_align_len,query_align_cov,subject_align_len,subject_align_cov,bit_score,taxa_pair_index,is_pair
0,16424,16564,0.930358,0.927849,0.927849,0.0,3,1483,1481,8,1481,0.998651,1474,0.993931,1161.0,741269,True
1,16424,16437,0.889191,0.881996,0.883784,0.0,6,1483,1477,6,1478,0.996628,1472,0.996615,958.0,741271,True
2,16507,16345,0.930499,0.935550,0.935867,0.0,1,1474,1473,1,1474,1.000000,1473,1.000000,1150.0,744513,True
3,16507,16258,0.898236,0.898236,0.898541,0.0,1,1474,1473,1,1474,1.000000,1473,1.000000,1001.0,744518,True
4,14004,2010,0.913142,0.896884,0.901902,0.0,31,1348,1333,16,1318,0.977745,1318,0.988747,953.0,614416,True


In [15]:
con.execute("""SELECT * FROM vp_taxa LIMIT 1""").df()

,taxa_index,ncbi_taxid,record_name,filepath,taxonomy,organism,bacdive_id,ogt_scraped_string,seq_16srRNA,len_16s,ogt,thermophile_label
0,5,1763543,NZ_BMNB01000001,./data/refseq/bacteria/GCF_014646235.1_ASM1464...,Bacteria Actinobacteria Micromonosporales Micr...,Verrucosispora sonchi,132025,28,TTGTTGGAGAGTTTGATCCTGGCTCAGGACGAACGCTGGCGGCGTG...,1517,28.0,False


In [10]:
con.execute("""SELECT * FROM vp_final LIMIT 1""").df().columns

Index(['thermo_protein_index', 'meso_protein_index',
       'local_gap_compressed_percent_id', 'scaled_local_query_percent_id',
       'scaled_local_symmetric_percent_id', 'local_E_value',
       'query_align_start', 'query_align_end', 'subject_align_end',
       'subject_align_start', 'query_align_len', 'query_align_cov',
       'subject_align_len', 'subject_align_cov', 'bit_score', 'thermo_index',
       'meso_index', 'prot_pair_index', 'meso_protein_int_index',
       'thermo_protein_int_index', 'taxa_pair_index',
       'local_gap_compressed_percent_id_16s',
       'scaled_local_query_percent_id_16s',
       'scaled_local_symmetric_percent_id_16s', 'query_align_cov_16s',
       'subject_align_cov_16s', 'bit_score_16s', 'm_ogt', 't_ogt',
       'ogt_difference', 'm_protein_seq', 't_protein_seq', 'm_protein_desc',
       't_protein_desc', 'm_protein_len', 't_protein_len'],
      dtype='object')